In [75]:
import pandas as pd
import numpy as np
import psycopg2
import requests
import websocket
import mplfinance as mpf

In [137]:
def fetch_intraday_data(ticker, interval='1min'):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}&outputsize=full&apikey=RV6W5BZQAXDT3ZVO'
    response = requests.get(url)
    data = response.json()
    # print(data)

    time_series = data.get(f'Time Series ({interval})', {})
    if not time_series:
        print(f"No data returned for {ticker}")
        return None
    df = pd.DataFrame(time_series).T
    df.index.rename('timestamp', inplace=True)
    pd.to_datetime(df.index)
    df = df.sort_index()
    df.columns = ['open', 'high', 'low', 'close', 'volume']
    df['ticker'] = ticker
    return df

In [138]:
df = fetch_intraday_data('MSFT')

No data returned for MSFT


In [123]:
df.head()

,1. open,2. high,3. low,4. close,5. volume,ticker
timestamp,,,,,,
2024-08-01 04:00:00,16.5000,16.5000,16.4600,16.4700,329,RIVN
2024-08-01 04:01:00,16.5000,16.5100,16.5000,16.5100,104,RIVN
2024-08-01 04:02:00,16.5600,16.5600,16.5600,16.5600,4,RIVN
2024-08-01 04:03:00,16.5000,16.5000,16.4700,16.4700,1687,RIVN
2024-08-01 04:04:00,16.4700,16.4700,16.4700,16.4700,200,RIVN


In [124]:
df.dtypes

1. open      object
2. high      object
3. low       object
4. close     object
5. volume    object
ticker       object
dtype: object

In [83]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df['open'] = df['open'].astype('float')
df['high'] = df['high'].astype('float')
df['low'] = df['low'].astype('float')
df['close'] = df['close'].astype('float')
df['volume'] = df['volume'].astype('float')
df['ticker'] = df['ticker'].astype('string')
mpf.plot(df, type='candle', volume=True, style='yahoo')

TypeError: Expect data.index as DatetimeIndex